# Chapter 45: Migration Strategies

---

## Introduction

Migrating an existing JavaScript codebase to TypeScript is one of the most common challenges teams face. Unlike greenfield projects where strict configurations can be enforced from day one, migrations require balancing type safety with business continuity. A poorly planned migration can halt feature development, introduce regressions, or create a "worst of both worlds" scenario where you have the verbosity of TypeScript without its safety guarantees.

This chapter provides battle-tested migration strategies used by organizations like Airbnb, Slack, and Microsoft to incrementally adopt TypeScript in large JavaScript codebases. We'll cover technical approaches (incremental compilation, JSDoc annotations, automated codemods) and organizational strategies (team training, CI/CD integration, strict mode rollouts) that allow you to ship code throughout the migration process.

---

## 45.1 Migrating from JavaScript to TypeScript

The key to a successful migration is incrementality. Freezing feature development to "convert everything" is a recipe for failure. Instead, TypeScript is designed to work alongside JavaScript, allowing gradual adoption file by file.

### 45.1.1 Initial Setup and Configuration

**Phase 1: Installation and Loose Configuration**

```typescript
// Step 1: Install TypeScript and Node types
// npm install --save-dev typescript @types/node

// Step 2: Initialize tsconfig.json with permissive settings
// npx tsc --init

// tsconfig.json - Migration starting point
{
  "compilerOptions": {
    // Allow JavaScript files to be compiled
    "allowJs": true,
    
    // Report errors in JavaScript files (optional during early migration)
    "checkJs": false,
    
    // Generate declaration files for JavaScript (if publishing library)
    "declaration": false,
    
    // Output directory
    "outDir": "./dist",
    
    // Allow default imports from modules with no default export
    "esModuleInterop": true,
    
    // Skip type checking of declaration files (faster builds during migration)
    "skipLibCheck": true,
    
    // Relaxed strictness for initial migration
    "strict": false,
    "noImplicitAny": false,
    
    // Module resolution
    "moduleResolution": "node",
    "target": "ES2020",
    "module": "ESNext"
  },
  "include": [
    "src/**/*"
  ],
  "exclude": [
    "node_modules",
    "dist"
  ]
}
```

**Explanation:**
- `allowJs: true`: Enables TypeScript compiler to process `.js` files alongside `.ts` files
- `checkJs: false`: Initially disabled to prevent overwhelming error reports in legacy JavaScript
- `strict: false`: Start loose; enable strict checks gradually as files are converted
- `noImplicitAny: false`: Allows JavaScript files to have implicit `any` types during transition

### 45.1.2 The Rename Strategy

The simplest migration approach is renaming files from `.js` to `.ts` and fixing type errors incrementally.

**File-by-File Migration Process:**

```typescript
// Before: utils.js (Legacy JavaScript)
function formatDate(date) {
  return new Date(date).toLocaleDateString();
}

function calculateTotal(items) {
  return items.reduce((sum, item) => sum + item.price, 0);
}

module.exports = { formatDate, calculateTotal };

// Step 1: Rename to utils.ts
// Step 2: Add types progressively

// After: utils.ts (Migrated TypeScript)
// Add type annotations only where necessary for clarity/safety
function formatDate(date: string | Date): string {
  return new Date(date).toLocaleDateString();
}

interface Item {
  price: number;
  // Other properties can remain implicit during early migration
}

function calculateTotal(items: Item[]): number {
  return items.reduce((sum, item) => sum + item.price, 0);
}

// Convert module.exports to ES exports or keep CommonJS with types
export { formatDate, calculateTotal };
```

**Migration Priority Queue:**

```typescript
// Prioritize migration order by:
// 1. Shared utilities (high impact, many consumers)
// 2. Bug-prone modules (complex logic, frequent errors)
// 3. New features (write in TypeScript from start)
// 4. Stable legacy code (low priority if working well)

// Example: Start with utility functions used across codebase
// src/utils/validation.js -> src/utils/validation.ts
// src/utils/formatting.js -> src/utils/formatting.ts
// src/config/constants.js -> src/config/constants.ts

// Then move to core business logic
// src/services/UserService.js -> src/services/UserService.ts

// Finally tackle UI components (often most complex due to framework types)
// src/components/UserProfile.jsx -> src/components/UserProfile.tsx
```

### 45.1.3 Using JSDoc for Gradual Typing

Before converting files to TypeScript, you can add type safety using JSDoc annotations. This provides IntelliSense and type checking without changing file extensions.

**JSDoc Type Annotations:**

```typescript
// utils.js (Still JavaScript, but type-checked)

/**
 * Formats a date string or Date object
 * @param {string | Date} date - The date to format
 * @returns {string} Formatted date string
 */
function formatDate(date) {
  return new Date(date).toLocaleDateString();
}

/**
 * Calculates total price from items
 * @param {Array<{price: number}>} items - Array of items with prices
 * @returns {number} Total sum
 */
function calculateTotal(items) {
  return items.reduce((sum, item) => sum + item.price, 0);
}

// Enable checkJs to get TypeScript errors in JavaScript files
// "checkJs": true in tsconfig.json

// Advanced: Import types from TypeScript files
/**
 * @typedef {import('./types').User} User
 */

/**
 * Processes a user object
 * @param {User} user - The user to process
 * @returns {Promise<User>}
 */
async function processUser(user) {
  // TypeScript knows the shape of user here
  return user;
}
```

**Benefits of JSDoc Approach:**
- No build step changes required initially
- Editors (VS Code) provide IntelliSense based on JSDoc
- Can migrate file-by-file without breaking imports
- Enables `checkJs: true` to catch errors before full conversion

**Limitations:**
- Verbose syntax compared to TypeScript
- No generics support as powerful as TypeScript
- Cannot use advanced features like mapped types or conditional types

---

## 45.2 Handling Third-Party Libraries

During migration, you'll encounter libraries without TypeScript definitions. Handling these gracefully prevents migration blockers.

### 45.2.1 Installing Community Types

```typescript
// Check if types exist in DefinitelyTyped
// npm search @types/library-name

// Example: Adding types for lodash
// npm install --save-dev @types/lodash

// Example: Adding types for express
// npm install --save-dev @types/express @types/node

// Usage remains the same, but now typed
import _ from 'lodash';

// TypeScript now knows the signature
const result = _.chunk([1, 2, 3, 4], 2); // Type: number[][]
```

**When Types Don't Exist:**

```typescript
// Option 1: Declare module wildcard (quick fix, no type safety)
// types/declarations.d.ts
declare module 'untyped-library' {
  const content: any;
  export default content;
}

// Option 2: Gradual typing with specific exports
declare module 'legacy-utils' {
  export function parseData(input: string): unknown;
  export function formatOutput(data: unknown): string;
  export const VERSION: string;
}

// Option 3: Augment existing modules with missing methods
// types/express-extensions.d.ts
import 'express';

declare global {
  namespace Express {
    interface Request {
      userId?: string; // Added by auth middleware
      session: SessionData;
    }
  }
}
```

### 45.2.2 Creating Declaration Files

For internal untyped modules or libraries you're wrapping, create `.d.ts` declaration files.

```typescript
// types/vendor-library.d.ts

// Declare a module that exports a class
declare module 'legacy-db-client' {
  export class DatabaseClient {
    constructor(connectionString: string);
    
    connect(): Promise<void>;
    disconnect(): Promise<void>;
    
    query<T = unknown>(sql: string, params?: unknown[]): Promise<T[]>;
    execute(sql: string, params?: unknown[]): Promise<{ rowsAffected: number }>;
  }
  
  export interface ConnectionConfig {
    host: string;
    port: number;
    database: string;
    username: string;
    password: string;
  }
}

// Declare global variables from scripts
declare global {
  interface Window {
    analytics?: {
      track(event: string, properties?: Record<string, unknown>): void;
    };
    legacyConfig?: {
      apiUrl: string;
      debug: boolean;
    };
  }
}

// Declare modules that import non-code assets
declare module '*.svg' {
  const content: string;
  export default content;
}

declare module '*.scss' {
  const classes: { [key: string]: string };
  export default classes;
}
```

**Explanation:**
- Declaration files (`.d.ts`) provide types without implementation
- `declare module` creates types for untyped npm packages
- `declare global` augments global objects like `Window` or `Process`
- Type-only declarations prevent "module not found" errors while allowing gradual typing

---

## 45.3 Migration Tooling

Manual migration is tedious and error-prone. Automated tools can handle the boilerplate while you focus on architecture.

### 45.3.1 ts-migrate

Airbnb's `ts-migrate` is a codemod toolkit for mass JavaScript-to-TypeScript conversion.

```bash
# Installation
npm install --save-dev ts-migrate

# Initialize (rename files, create tsconfig)
npx ts-migrate init src --aliases relative

# Rename files .js -> .ts (or .jsx -> .tsx)
npx ts-migrate rename src

# Add JSDoc types based on inference
npx ts-migrate migrate src

# Specific codemods
npx ts-migrate codemod js-to-ts src  # Convert JSDoc to TypeScript
npx ts-migrate codemod annotate src  // Add explicit any annotations to silence errors
```

**What ts-migrate Does:**

```typescript
// Before: utils.js
function getUser(id) {
  return fetch(`/api/users/${id}`).then(r => r.json());
}

// After ts-migrate annotate:
function getUser(id: any): any {
  return fetch(`/api/users/${id}`).then((r: any) => r.json());
}

// After ts-migrate migrate (with JSDoc):
/**
 * @param {string | number} id
 * @returns {Promise<any>}
 */
function getUser(id) {
  return fetch(`/api/users/${id}`).then(r => r.json());
}
```

**Post-Migration Cleanup Strategy:**

```typescript
// ts-migrate adds @ts-expect-error comments and any types
// Your job is to replace them incrementally:

// Step 1: Replace any with unknown for safety
function parseData(input: unknown): unknown {
  // Implementation
}

// Step 2: Add proper return types
async function getUser(id: string): Promise<User | null> {
  const response = await fetch(`/api/users/${id}`);
  if (!response.ok) return null;
  return response.json();
}

// Step 3: Remove @ts-expect-error as you fix underlying issues
// @ts-expect-error TS2345: Argument of type X is not assignable to Y
// -> Fix the type mismatch, then remove comment
```

### 45.3.2 TypeScript Compiler API for Custom Codemods

For large-scale specific transformations, write custom codemods using the TypeScript compiler API.

```typescript
// scripts/migrate-types.ts
import * as ts from 'typescript';
import * as fs from 'fs';

// Custom transformer: Convert PropTypes to TypeScript interfaces
function migratePropTypes(sourceFile: ts.SourceFile): ts.SourceFile {
  const transformer: ts.TransformerFactory<ts.SourceFile> = (context) => {
    const visit: ts.Visitor = (node) => {
      // Detect PropTypes definitions
      if (
        ts.isPropertyAssignment(node) &&
        ts.isIdentifier(node.name) &&
        node.name.text === 'propTypes'
      ) {
        // Transform to interface (simplified example)
        const interfaceDecl = ts.factory.createInterfaceDeclaration(
          [ts.factory.createToken(ts.SyntaxKind.ExportKeyword)],
          ts.factory.createIdentifier('Props'),
          undefined,
          undefined,
          [] // Property signatures would be extracted here
        );
        return interfaceDecl;
      }
      return ts.visitEachChild(node, visit, context);
    };
    return (node) => ts.visitNode(node, visit);
  };
  
  const result = ts.transform(sourceFile, [transformer]);
  return result.transformed[0];
}

// Usage
const fileName = 'Component.jsx';
const sourceCode = fs.readFileSync(fileName, 'utf8');
const sourceFile = ts.createSourceFile(
  fileName,
  sourceCode,
  ts.ScriptTarget.Latest,
  true
);

const migrated = migratePropTypes(sourceFile);
const printer = ts.createPrinter();
const result = printer.printFile(migrated);
fs.writeFileSync('Component.tsx', result);
```

### 45.3.3 ESLint with TypeScript for Migration Quality

Configure ESLint to prevent regression during migration.

```typescript
// .eslintrc.json - Migration configuration
{
  "root": true,
  "parser": "@typescript-eslint/parser",
  "plugins": ["@typescript-eslint"],
  "extends": [
    "eslint:recommended",
    "plugin:@typescript-eslint/recommended",
    "plugin:@typescript-eslint/recommended-requiring-type-checking"
  ],
  "parserOptions": {
    "project": "./tsconfig.json"
  },
  "rules": {
    // Error on explicit any in new files
    "@typescript-eslint/no-explicit-any": "error",
    
    // Warn on implicit any (for files not yet converted)
    "@typescript-eslint/no-implicit-any-catch": "warn",
    
    // Enforce type exports/imports
    "@typescript-eslint/consistent-type-imports": "error",
    
    // Prevent @ts-ignore (use @ts-expect-error with reason)
    "@typescript-eslint/ban-ts-comment": [
      "error",
      {
        "ts-ignore": "allow-with-description",
        "ts-expect-error": "allow-with-description",
        "minimumDescriptionLength": 10
      }
    ],
    
    // Relax during migration, stricten after
    "@typescript-eslint/explicit-function-return-type": "off",
    "@typescript-eslint/explicit-module-boundary-types": "off"
  },
  "overrides": [
    {
      // Relaxed rules for files not yet migrated
      "files": ["*.js"],
      "rules": {
        "@typescript-eslint/no-explicit-any": "off"
      }
    }
  ]
}
```

---

## 45.4 Strict Mode Adoption

Enabling strict mode all at once in a large codebase creates thousands of errors. Instead, enable strict checks incrementally.

### 45.4.1 Gradual Strictness Roadmap

```typescript
// tsconfig.json - Stage 1: Baseline (Permissive)
{
  "compilerOptions": {
    "strict": false,
    "noImplicitAny": false,
    "strictNullChecks": false,
    "strictFunctionTypes": false,
    "strictBindCallApply": false,
    "strictPropertyInitialization": false,
    "noImplicitThis": false,
    "alwaysStrict": false
  }
}

// Stage 2: Enable after 25% conversion
{
  "compilerOptions": {
    "strict": false,
    "noImplicitAny": true,      // Catch missing type annotations
    "strictNullChecks": false,  // Still relaxed for nulls
    "strictFunctionTypes": false
  }
}

// Stage 3: Enable after 50% conversion
{
  "compilerOptions": {
    "strict": false,
    "noImplicitAny": true,
    "strictNullChecks": true,   // Critical for safety
    "strictFunctionTypes": true,
    "strictBindCallApply": false
  }
}

// Stage 4: Full strict mode (after 80%+ conversion)
{
  "compilerOptions": {
    "strict": true,  // Enables all strict options
    "noUncheckedIndexedAccess": true,  // Extra strictness
    "exactOptionalPropertyTypes": true
  }
}
```

### 45.4.2 Per-File Strictness Overrides

Use comments to enable strict checks for specific files during transition.

```typescript
// @ts-check - Enable type checking in JavaScript files
// @ts-nocheck - Disable type checking for problematic files during migration

// At top of newly converted files, enable strictness:
// @ts-strict
import { User } from './types';

// This file now has strict checking even if tsconfig is loose
function processUser(user: User) {
  // Strict null checks apply here even if globally disabled
  return user.name.toUpperCase(); // Error if strictNullChecks enabled via comment
}
```

**Note:** The `@ts-strict` comment is not built into TypeScript, but you can achieve similar with project references:

```typescript
// tsconfig.strict.json
{
  "extends": "./tsconfig.json",
  "compilerOptions": {
    "strict": true
  },
  "include": [
    "src/converted/**/*"  // Only strict-check migrated files
  ]
}
```

### 45.4.3 Handling Implicit Any

The most common migration blocker is `noImplicitAny` violations.

```typescript
// Before: Implicit any (noImplicitAny: false)
function processData(data) {
  return data.map(item => item.value);
}

// After: Gradual typing approach
// Step 1: Explicit any (mark for future improvement)
function processData(data: any): any {
  return data.map((item: any) => item.value);
}

// Step 2: Partial typing
function processData(data: any[]): any[] {
  return data.map(item => item.value);
}

// Step 3: Full typing
interface DataItem {
  value: number;
  label: string;
}

function processData(data: DataItem[]): number[] {
  return data.map(item => item.value);
}
```

---

## 45.5 Team Adoption Strategies

Technical migration is only half the battle; team adoption determines long-term success.

### 45.5.1 Migration Timeline and Planning

```typescript
// Realistic migration timeline for medium codebase (50k+ lines)

// Week 1-2: Infrastructure
// - Setup TypeScript build pipeline
// - Configure loose tsconfig
// - Setup CI to check new files
// - Training session for team

// Week 3-6: Foundation Layer
// - Migrate shared utilities
// - Create type definitions for domain models
// - Setup strict checking for utils folder only

// Month 2-3: Feature Migration
// - Migrate 1 feature per developer per week
// - "No new JS" rule: All new code in TypeScript
// - Weekly type-safety review meetings

// Month 4-6: Strict Mode Rollout
// - Enable strictNullChecks
// - Fix type errors incrementally
// - Add runtime validation at boundaries

// Ongoing: Maintenance
// - Strict mode enabled globally
// - Type coverage monitoring
// - Regular dependency type updates
```

### 45.5.2 CI/CD Integration During Migration

```yaml
# .github/workflows/ci.yml
name: CI
on: [push]

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      
      # Type check only TypeScript files (ignore JS errors initially)
      - name: Type Check
        run: npx tsc --noEmit
        continue-on-error: true  # Don't block builds during early migration
      
      # Lint only TypeScript files with strict rules
      - name: Lint TypeScript
        run: npx eslint 'src/**/*.{ts,tsx}'
      
      # Track type coverage (optional but recommended)
      - name: Type Coverage
        run: |
          npx type-coverage --strict --min=80  # Require 80% type coverage
```

**Git Hooks for Prevention:**

```json
// package.json
{
  "husky": {
    "hooks": {
      "pre-commit": "lint-staged"
    }
  },
  "lint-staged": {
    "*.{ts,tsx}": [
      "eslint --fix",
      "bash -c 'tsc --noEmit'",  // Type check staged files
      "git add"
    ],
    "*.{js,jsx}": [
      "eslint --fix",
      "git add"
    ]
  }
}
```

### 45.5.3 Training and Documentation

```typescript
// Create internal migration guide: MIGRATION.md

// ## File Conversion Checklist

// When converting a JavaScript file to TypeScript:

// 1. Rename file extension (.js -> .ts, .jsx -> .tsx)
// 2. Add explicit return types to exported functions
// 3. Define interfaces for function parameters
// 4. Replace `require` with `import`
// 5. Add null checks for optional properties
// 6. Run type checker and fix errors
// 7. Add types for third-party libraries if missing
// 8. Update unit tests to use TypeScript (if applicable)

// ## Common Patterns

// Before (JS):
const user = await getUser(id);
console.log(user.name);

// After (TS):
interface User {
  id: string;
  name: string;
}

const user: User | null = await getUser(id);
if (!user) {
  throw new Error(`User ${id} not found`);
}
console.log(user.name);
```

---

## 45.6 Chapter Summary and Exercises

### Chapter Summary

This chapter covered practical strategies for adopting TypeScript in existing JavaScript projects:

1. **Incremental Migration**: Use `allowJs` to run TypeScript alongside JavaScript. Migrate file-by-file starting with shared utilities, not UI components.

2. **JSDoc Annotations**: Add type safety to JavaScript files using JSDoc before renaming to `.ts`, enabling `checkJs` for gradual error detection.

3. **Third-Party Types**: Install `@types` packages when available, create `.d.ts` declaration files for missing types, and use `declare module` for quick fixes.

4. **Automation**: Use `ts-migrate` for mass conversion, custom codemods for specific patterns, and ESLint to prevent regression.

5. **Strict Mode Rollout**: Enable strict checks incrementally (noImplicitAny → strictNullChecks → strict: true) rather than all at once.

6. **Team Adoption**: Enforce "TypeScript for new code" rules, use CI to track type coverage, and provide clear checklists for file conversion.

### Migration Checklist

```
┌─────────────────────────────────────────────────────────────────────┐
│                    JavaScript to TypeScript Migration Checklist      │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Setup Phase                                                       │
│   □ Install TypeScript and @types/node                              │
│   □ Create permissive tsconfig.json (allowJs: true, strict: false)  │
│   □ Configure build pipeline to handle .ts files                    │
│   □ Setup ESLint with TypeScript parser                             │
│                                                                     │
│   Initial Migration                                                 │
│   □ Rename shared utility files (.js -> .ts)                        │
│   □ Add JSDoc to remaining JS files for IntelliSense                │
│   □ Create type definitions for domain models                       │
│   □ Declare modules for untyped third-party libraries               │
│                                                                     │
│   Quality Gates                                                     │
│   □ Enable noImplicitAny after 25% conversion                       │
│   □ Enable strictNullChecks after 50% conversion                    │
│   □ Enable full strict mode after 80% conversion                    │
│   □ Setup CI to block new JS files (except config)                  │
│                                                                     │
│   Team Process                                                      │
│   □ All new features written in TypeScript                          │
│   □ Code review checklist for type safety                           │
│   □ Documentation for common migration patterns                     │
│   □ Training on strict mode benefits                                │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### Practical Exercises

**Exercise 1: Migration Setup**

Set up a migration configuration for a JavaScript Express application:

```typescript
// Tasks:
// 1. Create tsconfig.json that allows existing JS files
// 2. Configure it to check JS files but not fail builds
// 3. Setup path aliases to avoid relative path hell during migration
// 4. Create type declarations for Express app locals
```

**Exercise 2: JSDoc Conversion**

Convert this JavaScript file using JSDoc annotations first, then TypeScript:

```typescript
// services/UserService.js
class UserService {
  constructor(db) {
    this.db = db;
  }
  
  async findById(id) {
    const user = await this.db.users.findOne({ id });
    return user;
  }
  
  async create(userData) {
    if (!userData.email) {
      throw new Error('Email required');
    }
    return this.db.users.insert(userData);
  }
}

module.exports = UserService;

// Step 1: Add JSDoc types
// Step 2: Convert to TypeScript with proper interfaces
// Step 3: Add null checks and error handling types
```

**Exercise 3: Declaration Files**

Create proper TypeScript support for this untyped library:

```typescript
// legacy-analytics.js (third party, no types)
// Usage in codebase:
// import analytics from 'legacy-analytics';
// analytics.init({ apiKey: 'xxx' });
// analytics.track('page_view', { path: '/home' });
// analytics.setUser({ id: '123', traits: { name: 'John' } });

// Tasks:
// 1. Create legacy-analytics.d.ts
// 2. Define types for init, track, and setUser methods
// 3. Make traits and properties Record<string, unknown>
// 4. Ensure analytics can be imported as default or named export
```

**Exercise 4: Strict Mode Gradual Enablement**

Given this codebase with implicit any throughout:

```typescript
// Current tsconfig
{ "compilerOptions": { "strict": false } }

// Tasks:
// 1. Create a plan to enable noImplicitAny without breaking builds
// 2. Write a script to find all implicit any locations
// 3. Create a tsconfig.strict.json for new files only
// 4. Document the strategy for fixing "any" types over time
```

**Exercise 5: Migration Codemod**

Write a custom transformation to convert a specific pattern:

```typescript
// Convert this JavaScript pattern:
const User = {
  findById: async (id) => { /* ... */ },
  create: async (data) => { /* ... */ }
};

// To TypeScript class:
class UserRepository {
  async findById(id: string): Promise<User | null> { /* ... */ }
  async create(data: UserCreateInput): Promise<User> { /* ... */ }
}

// Use ts-migrate or TypeScript compiler API
```

---

## Next Chapter Preview

### Chapter 46: Latest TypeScript Features

Moving into Part XIV, we explore the cutting-edge features of TypeScript 5.x:

- **Decorators (Stage 3)**: The new standard decorators implementation replacing experimental decorators, with typing for class, method, and property decorators
- **`const` Type Parameters**: Using `const` assertions in generic constraints for literal type inference
- **`satisfies` Operator**: Type checking without widening, ensuring values meet interfaces while preserving specific literal types
- **Import Attributes and Assertions**: Type-safe import assertions for JSON modules and other resource types
- **Enum Improvements**: Modern enum behavior, const enums vs. regular enums, and alternatives (const objects with `as const`)
- **Module Resolution Enhancements**: `bundler` module resolution, package.json exports support, and ESM/CJS interoperability improvements
- **Performance Improvements**: Faster type checking, smarter caching, and reduced memory usage in large projects

This chapter will ensure you're leveraging the latest TypeScript capabilities for maximum type safety and developer experience.

---

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='44. common_pitfalls_and_how_to_avoid_them.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../14. typescript_5x_features/46. latest_typescript_features.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
